
## Simulation of the LHC 6L2 module with Wakis

| Result                                                                                                                                                                  | Mesh and parameters | Model                                                                                                                                                                                                                                                       |
| ----------------------------------------------------------------------------------------------------------------------------------------------------------------------- | ------------------- | ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| ![](https://codimd.web.cern.ch/uploads/upload_4494dc3d0aac49d88cb5b49e4926f53d.png) ![](https://codimd.web.cern.ch/uploads/upload_f9b4e53e2ddda4403d84acd70c58f670.png) |        ![](https://codimd.web.cern.ch/uploads/upload_7cc0b2faab05035ce9a14e3b3b0702ff.PNG)             | ![](https://codimd.web.cern.ch/uploads/upload_8571bf9c24f603c1f787f84e078d94f1.png) ![](https://codimd.web.cern.ch/uploads/upload_35a9cc226c5d945c2137c740db643c49.png) ![](https://codimd.web.cern.ch/uploads/upload_d4e0937c57ff04ac1a796a600798bd05.png) |


### Generate the STL files from STEP
> Needs to use cadquery venv

In [ ]:
# test cadquery import
import cadquery as cq
cq.__version__

In [ ]:
from wakis import geometry
stp_file = '012_VSMIO.stp'

In [ ]:
# Extract dictionaries with names of solids and materials
stl_solids = geometry.extract_solids_from_stp(stp_file)
print(f'Found {len(stl_solids)} solids in the STP file.')
print(stl_solids)
stl_colors = geometry.extract_colors_from_stp(stp_file)
print(f'Found {len(stl_colors)} colors in the STP file.')
print(stl_colors)
stl_materials = geometry.extract_materials_from_stp(stp_file)
print(f'Found {len(stl_materials)} materials in the STP file.')
print(stl_materials)
unit = geometry.get_stp_unit_scale(stp_file)
print(f'STP unit scale: {unit} m')

In [ ]:
stl_solids = geometry.generate_stl_solids_from_stp(stp_file)

### Combine STL files by material

In [ ]:
import pyvista as pv
import numpy as np
from wakis import geometry
stp_file = '012_VSMIO.stp'

# Extract dictionaries with names of solids and materials
stl_solids = geometry.extract_solids_from_stp(stp_file, path='stl')
print(f'Found {len(stl_solids)} solids in the STP file.')
print(stl_solids)
stl_colors = geometry.extract_colors_from_stp(stp_file)
print(f'Found {len(stl_colors)} colors in the STP file.')
print(stl_colors)
stl_materials = geometry.extract_materials_from_stp(stp_file)
print(f'Found {len(stl_materials)} materials in the STP file.')
print(stl_materials)
unit = geometry.get_stp_unit_scale(stp_file)
print(f'STP unit scale: {unit} m')

In [ ]:
materials = np.unique(list(stl_materials.values()))
print(f'Unique materials: {materials}')

colors = np.unique(list(stl_colors.values()), axis=0)
print(f'Unique colors: {colors}')

new_stl_colors = {}

In [ ]:
# Combine STL files by material
material = '316l'
aux = None
for key, mat in stl_materials.items():
    if mat.lower() == material.lower():
        color = stl_colors[key]
        print(f"Adding solid: {key.split('_')[0]} with material: {mat}")
        if aux is None:
            aux = pv.read(stl_solids[key]).triangulate()
        else:
            surf = pv.read(stl_solids[key]).triangulate()
            aux = aux + surf
surf = aux
del aux
surf.save(f'012_LHC_WVM_6L2-{material}.stl')
surf.plot(color=color, silhouette=True)
new_stl_colors[material] = color

In [ ]:
material = 'cube'
aux = None
for key, mat in stl_materials.items():
    if mat.lower() == material.lower():
        print(f"Adding solid: {key.split('_')[0]} with material: {mat}")
        if aux is None:
            color = stl_colors[key]
            aux = pv.read(stl_solids[key]).triangulate()
        else:
            surf = pv.read(stl_solids[key]).triangulate()
            aux = aux + surf
surf = aux
del aux
surf.save(f'012_LHC_WVM_6L2-{material}.stl')
surf.plot(color=color, silhouette=True)
new_stl_colors[material] = color

In [ ]:
material = 'cucamera'
aux = None
for key, mat in stl_materials.items():
    if mat.lower() == material.lower():
        print(f"Adding solid: {key.split('_')[0]} with material: {mat}")
        if aux is None:
            color = stl_colors[key]
            aux = pv.read(stl_solids[key]).triangulate()
        else:
            surf = pv.read(stl_solids[key]).triangulate()
            aux = aux + surf
surf = aux
del aux
surf.save(f'012_LHC_WVM_6L2-{material}.stl')
surf.plot(color=color, silhouette=True)
new_stl_colors[material] = color

In [ ]:
material = 'martensite'
aux = None
for key, mat in stl_materials.items():
    if mat.lower() == material.lower():
        print(f"Adding solid: {key.split('_')[0]} with material: {mat}")
        if aux is None:
            color = stl_colors[key]
            aux = pv.read(stl_solids[key]).triangulate()
        else:
            surf = pv.read(stl_solids[key]).triangulate()
            aux = aux + surf
surf = aux
del aux
surf.save(f'012_LHC_WVM_6L2-{material}.stl')
surf.plot(color=color, silhouette=True)
new_stl_colors[material] = color

In [ ]:
print('New STL colors dictionary:')
print(new_stl_colors)

### Generate simulation mesh

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pyvista as pv

from wakis import SolverFIT3D
from wakis import GridFIT3D 
from wakis import WakeSolver
from wakis import geometry

%matplotlib ipympl

In [ ]:
## Input dictionaries
stp_file = '012_VSMIO.stp'
basename = '012_LHC_WVM_6L2'
stp_materials= geometry.extract_materials_from_stp(stp_file)
stp_colors = geometry.extract_colors_from_stp(stp_file)
materials = np.unique(list(stp_materials.values()))[:-1]
colors = np.unique(list(stp_colors.values()), axis=0)
print(f'Materials in STEP model: {materials}')
print(f'Colors in STEP model: {colors}')

stl_solids, stl_materials = {}, {}
for material in materials:
    print(f'Processing material: {material}')
    stl_solids[material] = f'{basename}-{material}.stl'
    stl_materials[material] = material

stl_colors = {'316l': [1.,1.,1.], 
            'cube':  [0.82745099, 0.698039, 0.49019599],  
            'cucamera': [0.82352901, 0.82352901, 1. ],  
            'martensite': [1.,1.,1.], }
print(stl_solids)
print(stl_colors)

Materials in STEP model: ['316l' 'cube' 'cucamera' 'martensite']
Colors in STEP model: [[0.82352901 0.82352901 1.        ]
 [0.82745099 0.698039   0.49019599]
 [1.         1.         1.        ]]
Processing material: 316l
Processing material: cube
Processing material: cucamera
Processing material: martensite
{'316l': '012_LHC_WVM_6L2-316l.stl', 'cube': '012_LHC_WVM_6L2-cube.stl', 'cucamera': '012_LHC_WVM_6L2-cucamera.stl', 'martensite': '012_LHC_WVM_6L2-martensite.stl'}
{'316l': [1.0, 1.0, 1.0], 'cube': [0.82352901, 0.82352901, 1.0], 'cucamera': [0.82745099, 0.698039, 0.49019599], 'martensite': [1.0, 1.0, 1.0]}


In [10]:
import pyvista as pv

def test_plot_solids_with_widgets_from_paths(
    solid_paths,
    stl_colors=None,
    opacity=1.0,
    inactive_opacity=0.1,
):
    """
    Standalone test function using a dict {name: 'file.stl'}.

    Parameters
    ----------
    solid_paths : dict[str, str]
        name -> STL file path

    opacity : float
        Opacity when a solid is highlighted.

    inactive_opacity : float
        Opacity when a solid is not highlighted.
    """

    pl = pv.Plotter()

    solid_state = {}

    # Load meshes and add them once; create a separate silhouette actor
    for name, path in solid_paths.items():
        mesh = pv.read(path)
        color = stl_colors.get(name, "lightgray") if stl_colors else "lightgray"

        actor = pl.add_mesh(
            mesh,
            color=color,
            name=name,
            opacity=inactive_opacity,
            silhouette=False,  # don't let add_mesh spawn its own silhouette actor
        )

        # Explicit silhouette actor (initially hidden)
        sil = pl.add_silhouette(
            mesh,
            color='black',
            line_width=3.0,
        )
        sil.SetVisibility(False)

        solid_state[name] = {
            "actor": actor,
            "silhouette": sil,
            "active_opacity": opacity,
            "inactive_opacity": inactive_opacity,
            "checked": False,
            "highlight": False,
            "button": None,
        }

    pl.set_background("white")
    pl.add_axes()

    # --- beam & integration path geometry (centered along z, at x=y=0) ---
    x_min, x_max, y_min, y_max, z_min, z_max = pl.bounds
    z_center = 0.5 * (z_min + z_max)
    z_height = z_max - z_min

    # choose a very thin radius relative to the scene size
    scene_extent = max(x_max - x_min, y_max - y_min, z_height)
    radius = 0.005 * scene_extent if scene_extent > 0 else 1e-3

    beam_cyl = pv.Cylinder(
        center=(0.0, 0.0, z_center),
        direction=(0.0, 0.0, 1.0),
        height=z_height,
        radius=radius,
    )

    path_cyl = pv.Cylinder(
        center=(0.0, 0.0, z_center),
        direction=(0.0, 0.0, 1.0),
        height=z_height,
        radius=radius,
    )

    beam_actor = pl.add_mesh(
        beam_cyl,
        color="orange",
        name="beam",
        opacity=1.0,
        smooth_shading=True,
    )
    beam_actor.SetVisibility(False)

    path_actor = pl.add_mesh(
        path_cyl,
        color="blue",
        name="integration_path",
        opacity=1.0,
        smooth_shading=True,
    )
    path_actor.SetVisibility(False)

    # -----------------------------------------------------
    #            ADAPTIVE UI SCALING (resolution-aware)
    # -----------------------------------------------------
    _, h = pl.window_size
    ui_scale = h / 800.0

    box_size  = max(16, int(20 * ui_scale))
    font_size = max(8,  int(12 * ui_scale))
    padding   = int(5 * ui_scale)

    base_x = int(10 * ui_scale)
    base_y = h // 2
    dy = box_size + padding

    # -----------------------------------------------------
    #              VISUAL STATE APPLICATION
    # -----------------------------------------------------
    def _apply_visual_state(state):
        """Apply opacity + silhouette visibility based on state['highlight']."""
        if state["highlight"]:
            state["actor"].GetProperty().SetOpacity(state["active_opacity"])
            state["silhouette"].SetVisibility(True)
        else:
            state["actor"].GetProperty().SetOpacity(state["inactive_opacity"])
            state["silhouette"].SetVisibility(False)

    # -----------------------------------------------------
    #                   CHECKBOX LOGIC
    # -----------------------------------------------------
    master_on = True

    def make_toggle_callback(name):
        def _toggle(checked):
            state = solid_state[name]
            state["checked"] = bool(checked)
            state["highlight"] = state["checked"]
            _apply_visual_state(state)
        return _toggle

    def master_callback(checked):
        nonlocal master_on
        master_on = bool(checked)

        for state in solid_state.values():
            # set all individual checkboxes to match master
            state["checked"] = master_on
            state["highlight"] = master_on and state["checked"]

            btn = state.get("button")
            if btn is not None:
                rep = btn.GetRepresentation()
                if hasattr(rep, "SetState"):
                    rep.SetState(1 if state["checked"] else 0)

            _apply_visual_state(state)

    # master checkbox (start with all dimmed: value=False)
    master_button = pl.add_checkbox_button_widget(
        master_callback,
        value=False,
        position=(base_x, base_y),
        size=box_size,
    )
    pl.add_text(
        "All on",
        position=(base_x + box_size + padding, base_y),
        font_size=font_size,
    )

    # per-solid checkboxes
    for i, name in enumerate(solid_state.keys()):
        y = base_y - (i + 2) * dy

        btn = pl.add_checkbox_button_widget(
            make_toggle_callback(name),
            value=False,
            position=(base_x, y),
            size=box_size,
        )
        solid_state[name]["button"] = btn

        pl.add_text(
            name,
            position=(base_x + box_size + padding, y),
            font_size=font_size,
        )

    # --- right-side UI: center-right checkboxes for beam & integration path ---
    w, h = pl.window_size  # you already used h; we now also need w

    right_box_size  = box_size      # keep same size as left panel
    right_font_size = font_size
    right_dy        = dy

    # place the checkboxes near the right edge, vertically centered
    base_x_right = int(w - right_box_size - 10 * ui_scale)

    num_right_rows = 2  # beam + integration path
    total_right_height = num_right_rows * right_dy
    base_y_right = int((h - total_right_height) / 2)

    text_offset_x = int(80 * ui_scale)  # distance of label to the LEFT of the checkbox

    def toggle_beam(checked):
        beam_actor.SetVisibility(bool(checked))

    def toggle_path(checked):
        path_actor.SetVisibility(bool(checked))

    # beam checkbox (top)
    y_beam = base_y_right
    pl.add_checkbox_button_widget(
        toggle_beam,
        value=False,
        position=(base_x_right, y_beam),
        size=right_box_size,
    )
    pl.add_text(
        "beam",
        position=(base_x_right - text_offset_x, y_beam),
        font_size=right_font_size,
    )

    # integration path checkbox (below beam)
    y_path = base_y_right + right_dy
    pl.add_checkbox_button_widget(
        toggle_path,
        value=False,
        position=(base_x_right, y_path),
        size=right_box_size,
    )
    pl.add_text(
        "integration path",
        position=(base_x_right - text_offset_x, y_path),
        font_size=right_font_size,
    )


    pl.show()


In [11]:
test_plot_solids_with_widgets_from_paths(stl_solids, stl_colors=stl_colors)

Widget(value='<iframe src="http://localhost:44957/index.html?ui=P_0x794ea2f4b550_1&reconnect=auto" class="pyvi…

In [ ]:
pl = pv.Plotter()
pl.window_size